# Iceberg Tracking
This notebook documents the components and the pipeline of the iceberg tracking system, including data preparation, detection, tracking and visualization steps.

## Preparation

1. Create a directory with the name of your dataset in: ``data/``
2. Store the images in: ``data/<dataset_name>/images/raw/``
   - Tip: To avoid processing specific/irrelevant areas of the images, create a masking image with blacked out areas and store it in: ``data/<dataset_name>/images/mask.<image_format>``
3. Create a ground truth dataset with annotated detections that follows the MOT format and store the file in: ``data/<dataset_name>/annotations/gt.txt``
   - The MOT (Multiple Object Tracking) Challenge uses a standardized CSV format for ground truth annotations where each line represents one object instance, containing ten comma-separated values:
     ```
     <frame>, <id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, <x>, <y>, <z>
     _MG_17310,1,161.15,1087.59,247.32,90.46,0.9999,-1,-1,-1
     _MG_17310,2,2113.04,2085.49,248.25,97.31,0.9985,-1,-1,-1
     ```
   - For more information see [MOT Challenge](https://motchallenge.net/instructions/).

## Components
The tracking-pipeline consists of 5 components:

- `IcebergEmbeddingsTrainer`: To capture iceberg appearance similarity.
- `IcebergFeatureExtractor`: For extracting similarity features.
- `IcebergDetector`: For detecting icebergs in images.
- `IcebergTracker`: For tracking detected icebergs across image frames.
- `Visualizer`: For visualizing the results.

## Pipeline

#### Step 1: Import Required Modules
Imports all necessary custom classes for the iceberg tracking pipeline. 

In [ ]:
from embedding import IcebergEmbeddingsConfig, IcebergEmbeddingsTrainer
from detection import IcebergDetectionConfig, IcebergDetector
from tracking import IcebergTrackingConfig, IcebergTracker
from utils.feature_extraction import get_gt_thresholds
from utils.visualize import Visualizer

#### Step 2: Define Dataset Parameters
To use any of the these components, you must configure the `dataset` and `image_format` parameter. Various additional configuration options can be configured via the corresponding config-class for a component or straight via parametrization. For further configuration details, please refer to the docstrings and inline comments in the code.

In [ ]:
dataset = "hill_2min_2023-08"   # Name of the dataset stored in `data/` containing the iceberg images
image_format = "JPG"            # The file extension of the images

#### Step 3: Generate Iceberg Embeddings
The `IcebergEmbeddingsTrainer`-class implements a Siamese Neural Network training pipeline for learning and generating embeddings that capture iceberg appearance similarity. The pipeline uses Vision Transformers as the backbone architecture and trains on pairs of iceberg crops extracted from the `gt.txt`-file to learn discriminative features for iceberg tracking and identification tasks.

In [ ]:
# You only have to run this embedding pipeline for the gt-dataset once
config = IcebergEmbeddingsConfig(dataset=dataset, image_format=image_format)
trainer = IcebergEmbeddingsTrainer(config=config)
trainer.run_complete_pipeline()

#### Step 4: Detect Icebergs
The `IcebergDetector`-class adapts a pre-trained Faster R-CNN model with ResNet50 backbone and Feature Pyramid Network (FPN) using PyTorch. The detection pipeline consists of the following steps:

1. Preprocessing: Apply masking to specific areas that are not relevant to the images by setting masked pixels to black.
2. Training: K-fold cross-validation with early stopping and SGD optimizer
3. Prediction: Combination of Multi-scale and Sliding window detection with overlapping windows to provide better detection coverage and accuracy for different iceberg sizes
4. Postprocessing: Remove detections that touch image edges or masked regions, remove detections that are below/above a specified threshold and generate embeddings for final detections


In [ ]:
config = IcebergDetectionConfig(dataset=dataset, image_format=image_format)
detector = IcebergDetector(config=config)
detector.train()
detector.predict()

#### Step 5: Extract GT-Features Thresholds
The `IcebergFeatureExtractor`-class provides functionality for extracting and analyzing similarity features of icebergs across images frames in ground truth data for tracking purposes. It handles statistical thresholds from ground truth data in order to use them for tracking. The extraction pipeline consists of the following steps:

1. Loading pre-computed iceberg embeddings of ground truth icebergs
2. Computation of similarity features: appearance (based on embedding model), distance and size
3. Normalize features and computes thresholds

In [ ]:
# If you want to use user-defined threshold, you can skip this step
thresholds = get_gt_thresholds(dataset, image_format)

#### Step 6: Track Icebergs
The `IcebergTracker`-class is responsible for associating detected icebergs across consecutive frames to maintain a consistent track for each individual iceberg based on the detection data. The thresholds and the weighting of the 3 similarity features and the final score can be configured. The tracking algorithm is divided into the following phases:

1. Loading pre-computed iceberg embeddings of detected icebergs
2. Compute similarities between icebergs in consecutive frames based on appearance, distance and size
3. Preselect candidates that fulfill the threshold similarity requirements based on ground truth data (or user-defined thresholds)
4. Match icebergs that are each other’s highest-scoring similarity partner
5. Postprocess results e.g. filter out too short iceberg tracks


In [ ]:
tracking_config = IcebergTrackingConfig(dataset, image_format, thresholds=thresholds)
tracker = IcebergTracker(tracking_config)
tracker.track()

#### Step 7: Visualize Results
The `Visualizer`-class provides visualization functionalities for the iceberg detection (random colors) or tracking data (consistent color mapping for object tracking across frames), including annotation rendering and video generation capabilities. The following annotation options are possible:

- Draw IDs: Draw iceberg ID next to the corresponding iceberg
- Draw boxes: Draw bounding boxes around icebergs
- Draw contours: Draw the icebergs contours (outlines) of each iceberg, using the Segment Anything Model (SAM)
- Draw masks: Fill the iceberg area in a transparent color

In [ ]:
visualizer = Visualizer(dataset=dataset, image_format=image_format, annotation_source="tracking")
visualizer.annotate_icebergs(draw_ids=True, draw_boxes=True, draw_contours=True, draw_masks=True)